<a href="https://colab.research.google.com/github/liviubobocu/eeml/blob/main/mechanistic_interpretability_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EEML Mechanistic Interpretability Workshop
Arthur Conmy and Federico Barbero

*This notebook heavily borrows from the ARENA tutorial by Callum McDougall -- if you like this tutorial, we recommend looking further at the [ARENA material](https://arena-chapter1-transformer-interp.streamlit.app/[1.2]_Intro_to_Mech_Interp)*.

For those that feel like they could brush up on Transformers, we recommend [this great video tutorial](https://www.youtube.com/watch?v=bOYE6E8JrtU) by Neel Nanda. A lot of the material in this notebook has been built by Neel so a huge shoutout to him.

The tutorial is designed to introduce you to the core concepts of mechanistic interpretability using the [TransformerLens](https://github.com/TransformerLensOrg/TransformerLens) library. We thank the developers of this library for their efforts.


Most of the sections are constructed in the following way:

1. A particular feature of TransformerLens is introduced.
2. You are given an exercise, in which you have to apply the feature.

Each exercise will have a difficulty and importance rating out of 5, as well as an estimated maximum time you should spend on these exercises and sometimes a short annotation. You should interpret the ratings & time estimates relatively (e.g. if you find yourself spending about 50% longer on the exercises than the time estimates, adjust accordingly).

## Content & Learning Objectives

### 1️⃣ TransformerLens: Introduction

This section is designed to get you up to speed with the TransformerLens library. You'll learn how to load and run models, and learn about the shared architecture template for all of these models.

##### Learning Objectives

> - Load and run a `HookedTransformer` model
> - Understand the basic architecture of these models
> - Use the model's tokenizer to convert text to tokens, and vice versa
> - Know how to cache activations, and to access activations from the cache
> - Use `circuitsvis` to visualise attention heads

### 2️⃣ TransformerLens: Hooks

Next, you'll learn about hooks, which are a great feature of TransformerLens allowing you to access and intervene on activations within the model. We will mainly focus on the basics of hooks and using them to access activations. You will also build some tools to perform logit attribution within your model, so you can identify which components are responsible for your model's performance on certain tasks.

##### Learning Objectives

> - Understand what hooks are, and how they are used in TransformerLens
> - Use hooks to access activations, process the results, and write them to an external tensor
> - Build tools to perform attribution, i.e. detecting which components of your model are responsible for performance on a given task
> - Understand how hooks can be used to perform basic interventions like **ablation**

### 3️⃣ **Bonus:** Replicating [Golden Gate Claude](https://www.anthropic.com/news/golden-gate-claude)!

##### Learning Objectives:

Finally, you will be replicating one of the first impressive results from the field of mechanistic interpretability: the "Golden Gate Claude" experiment. You will learn how to take an LLM and *steer* it such that it loves to talk about the Golden Gate bridge and San Francisco!

> - Work with an LLM finetuned for chat interactions (like ChatGPT, Gemini, Claude, ...)
> - Get activations from the inside of a chat LLM
> - Do a small-scale replication of Anthropic's [Golden Gate Claude](https://www.anthropic.com/news/golden-gate-claude) famous result


## Setup code

If you get an error like this:

```
ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
```

Select Runtime > Restart (also discoverable with CTRL+Shift+P)

*Please run the code in the cells below to setup the notebook*

In [ ]:
# @title
import os
import sys
from pathlib import Path

import pkg_resources

IN_COLAB = "google.colab" in sys.modules

chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
branch = "main"

# Install dependencies
installed_packages = [pkg.key for pkg in pkg_resources.working_set]
if "transformer-lens" not in installed_packages:
    %pip install transformer_lens==2.16.1 einops eindex-callum jaxtyping git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

# Get root directory, handling 3 different cases: (1) Colab, (2) notebook not in ARENA repo, (3) notebook in ARENA repo
root = (
    "/content"
    if IN_COLAB
    else "/root"
    if repo not in os.getcwd()
    else str(next(p for p in Path.cwd().parents if p.name == repo))
)

if Path(root).exists() and not Path(f"{root}/{chapter}").exists():
    if not IN_COLAB:
        !sudo apt-get install unzip
        %pip install jupyter ipython --upgrade

    if not os.path.exists(f"{root}/{chapter}"):
        !wget -P {root} https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/{branch}.zip
        !unzip {root}/{branch}.zip '{repo}-{branch}/{chapter}/exercises/*' -d {root}
        !mv {root}/{repo}-{branch}/{chapter} {root}/{chapter}
        !rm {root}/{branch}.zip
        !rmdir {root}/ARENA_3.0-{branch}


if f"{root}/{chapter}/exercises" not in sys.path:
    sys.path.append(f"{root}/{chapter}/exercises")

os.chdir(f"{root}/{chapter}/exercises")

/tmp/ipython-input-1-3994548384.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
# @title
import torch as t
assert t.cuda.is_available(), "You are not connected to a GPU. Please use Runtime > Change Runtime Type and choose e.g. T4"
device = "cuda:0"

import functools
import sys
from pathlib import Path
from typing import Callable
import circuitsvis as cv
import einops
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from eindex import eindex
from IPython.display import display
from jaxtyping import Float, Int
from torch import Tensor
from tqdm import tqdm
from transformer_lens import (
    ActivationCache,
    FactoredMatrix,
    HookedTransformer,
    HookedTransformerConfig,
    utils,
)
from transformer_lens.hook_points import HookPoint
import gc
import json
import markdown
from huggingface_hub import hf_hub_download
from IPython.display import HTML

# Make sure exercises are in the path
chapter = "chapter1_transformer_interp"
section = "part2_intro_to_mech_interp"
root_dir = next(p for p in Path.cwd().parents if (p / chapter).exists())
exercises_dir = root_dir / chapter / "exercises"
section_dir = exercises_dir / section

import part2_intro_to_mech_interp.tests as tests
from plotly_utils import hist, imshow, plot_comp_scores, plot_logit_attribution, plot_loss_difference

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

MAIN = __name__ == "__main__"

# 1️⃣ TransformerLens: Introduction

> ##### Learning Objectives
>
> - Load and run a `HookedTransformer` model
> - Understand the basic architecture of these models
> - Use the model's tokenizer to convert text to tokens, and vice versa
> - Know how to cache activations, and to access activations from the cache
> - Use `circuitsvis` to visualise attention heads

## Introduction

The goal of mechanistic interpretability is to take a trained model and reverse engineer the algorithms the model learned during training from its weights. It is a fact about the world today that we have computer programs that can essentially speak English at a human level (GPT-3 was announced in 2020, PaLM in 2022, many more LLMs since then), and yet we have no idea how they work nor how to write one ourselves. The goal of mechanistic interpretability is to try to fix this! It is a very young and growing field, and there are a *lot* of open problems - if you would like to help, please try working on one! **For a list of open problems feel free to check out this [doc](https://docs.google.com/document/d/1WONBzNqfKIxERejrrPlQMyKqg7jSFW92x5UMXNrMdPo/edit#) to figure out where to start.**

There's a lot of excellent infrastructure like HuggingFace and DeepSpeed to *use* or *train* models, but very little to dig into their internals and reverse engineer how they work. **The TransformerLens library tries to solve that**, and to make it easy to get into the field even if you don't work at an industry org with real infrastructure! The core features were heavily inspired by [Anthropic's excellent Garcon tool](https://transformer-circuits.pub/2021/garcon/index.html) -- credit to Nelson Elhage and Chris Olah for building Garcon!

The point of this library is to keep the gap between having an experiment idea and seeing the results as small as possible, to make it easy for **research to feel like play** and to enter a flow state. This notebook demonstrates how the library works and how to use it, but if you want to see how well it works for exploratory research, check out [Neel Nanda's notebook analysing Indirect Objection Identification](https://github.com/neelnanda-io/TransformerLens/blob/main/Exploratory_Analysis_Demo.ipynb) or [Neel Nanda's recording of himself doing research](https://www.youtube.com/watch?v=yo4QvDn-vsU)!

## Loading and Running Models

TransformerLens comes loaded with >40 open source GPT-style models. You can load any of them in with `HookedTransformer.from_pretrained(MODEL_NAME)`. For this demo notebook we'll look at GPT-2 Small, an 80M parameter model, see the Available Models section for info on the rest.

In [ ]:
gpt2_small: HookedTransformer = HookedTransformer.from_pretrained("gpt2-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


### HookedTransformerConfig

Alternatively, you can define a config object, then call `HookedTransformer.from_config(cfg)` to define your model. This is particularly useful when you want to have finer control over the architecture of your model. We'll see an example of this in the next section, when we define an attention-only model to study induction heads.

Even if you don't define your model in this way, you can still access the config object through the `cfg` attribute of the model.

### 💻 Exercise - inspect your model

> ```yaml
> Difficulty: 🔴⚪⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
> ```

Use `gpt2_small.cfg` to find the following, for your GPT-2 Small model:

* Number of layers
* Number of heads per layer
* Maximum context window

You might have to check out the documentation page for some of these. If you're in VSCode then you can reach it by right-clicking on `HookedTransformerConfig` and choosing "Go to definition". If you're in Colab, then you can read the [GitHub page](https://github.com/neelnanda-io/TransformerLens).

<details>
<summary>Answer</summary>

The following parameters in the config object give you the answers:

```
cfg.n_layers == 12
cfg.n_heads == 12
cfg.n_ctx == 1024
```

</details>

### Running your model

Models can be run on a single string or a tensor of tokens (shape: `[batch, position]`, all integers). The possible return types are:

* `"logits"` (shape `[batch, position, d_vocab]`, floats),
* `"loss"` (the cross-entropy loss when predicting the next token),
* `"both"` (a tuple of `(logits, loss)`)
* `None` (run the model, but don't calculate the logits - this is faster when we only want to use intermediate activations)

In [ ]:
model_description_text = """## Loading Models

HookedTransformer comes loaded with >40 open source GPT-style models. You can load any of them in with `HookedTransformer.from_pretrained(MODEL_NAME)`. Each model is loaded into the consistent HookedTransformer architecture, designed to be clean, consistent and interpretability-friendly.

For this demo notebook we'll look at GPT-2 Small, an 80M parameter model. To try the model the model out, let's find the loss on this paragraph!"""

loss = gpt2_small(model_description_text, return_type="loss")
print("Model loss:", loss)

Model loss: tensor(4.3443, device='cuda:0')


<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">Model loss: tensor(4.3443, device='cuda:0')</pre>

## Transformer architecture

The key picture to keep in your head when thinking about the internals of tranformers is this one:

![Anthropic_Circuit_Diagram](https://img2024.cnblogs.com/blog/1797571/202503/1797571-20250325133836129-771431847.png)
*From Anthropic's [Mathematical Framework For Transformer Circuits paper](https://transformer-circuits.pub/2021/framework/index.html)*.

Do not worry about not understanding every part of the transformer in detail! Part of the reason to do workshops with colabs is so that you can help investigate models to figure out confusing details. (Also ask LLMs such as OpenAI o3 or Gemini 2.5 Pro or Claude 4 to help clarify details!)

One of the key bits of unusual terminology that Anthropic use in their [Mathematical Framework For Transformer Circuits paper](https://transformer-circuits.pub/2021/framework/index.html) is the reference to the internal current state of the activations as the **Residual Stream**. Each value $x_0, x_i, x_{i+1}, ...$ on the central vertical line in the diagram above is one value of the residual stream. We will use the phrase "residual stream" throughout this tutorial.

Now, back to TransformerLens. The `HookedTransformer` class is a somewhat adapted GPT-2 architecture, but is computationally identical. The most significant changes are to the internal structure of the attention heads:

* The weights `W_K`, `W_Q`, `W_V` mapping the residual stream to queries, keys and values are 3 separate matrices, rather than big concatenated one.
* The weight matrices `W_K`, `W_Q`, `W_V`, `W_O` and activations have separate `head_index` and `d_head` axes, rather than flattening them into one big axis.
    * The activations all have shape `[batch, position, head_index, d_head]`.
    * `W_K`, `W_Q`, `W_V` have shape `[head_index, d_model, d_head]` and `W_O` has shape `[head_index, d_head, d_model]`
* **Important - we generally follow the convention that weight matrices multiply on the right rather than the left.** In other words, they have shape `[input, output]`, and we have `new_activation = old_activation @ weights + bias`.
    * Click the dropdown below for examples of this, if it seems unintuitive.

<details>
<summary>Examples of matrix multiplication in our model</summary>

* **Query matrices**
    * Each query matrix `W_Q` for a particular layer and head has shape `[d_model, d_head]`.
    * So if a vector `x` in the residual stream has length `d_model`, then the corresponding query vector is `x @ W_Q`, which has length `d_head`.
* **Embedding matrix**
    * The embedding matrix `W_E` has shape `[d_vocab, d_model]`.
    * So if `A` is a one-hot-encoded vector of length `d_vocab` corresponding to a particular token, then the embedding vector for this token is `A @ W_E`, which has length `d_model`.

</details>

The actual code is a bit of a mess, as there's a variety of Boolean flags to make it consistent with the various different model families in TransformerLens - to understand it and the internal structure, I instead recommend reading the code in [CleanTransformerDemo](https://colab.research.google.com/github/neelnanda-io/TransformerLens/blob/clean-transformer-demo/Clean_Transformer_Demo.ipynb).

### Parameters and Activations

It's important to distinguish between parameters and activations in the model.

* **Parameters** are the weights and biases that are learned during training.
    * These don't change when the model input changes.
    * They can be accessed directly from the model, e.g. `model.W_E` for the embedding matrix.
* **Activations** are temporary numbers calculated during a forward pass, that are functions of the input.
    * We can think of these values as only existing for the duration of a single forward pass, and disappearing afterwards.
    * We can use hooks to access these values during a forward pass (more on hooks later), but it doesn't make sense to talk about a model's activations outside the context of some particular input.
    * Attention scores and patterns are activations (this is slightly non-intuitve because they're used in a matrix multiplication with another activation).
    * All values of the residual stream are activations, for example.

The link below shows a diagram of a single layer (called a `TransformerBlock`) for an attention-only model with no biases. Each box corresponds to an **activation** (and also tells you the name of the corresponding hook point, which we will eventually use to access those activations). The red text below each box tells you the shape of the activation (ignoring the batch dimension). Each arrow corresponds to an operation on an activation; where there are **parameters** involved these are labelled on the arrows.

[Link to diagram](https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/small-merm.svg)

The next link is to a diagram of a `TransformerBlock` with full features (including biases, layernorms, and MLPs). Don't worry if not all of this makes sense at first - we'll return to some of the details later. As we work with these transformers, we'll get more comfortable with their architecture.

[Link to diagram](https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/full-merm.svg)

A few shortcuts to make your lives easier when using these models:

* You can index weights like `W_Q` directly from the model via e.g. `model.blocks[0].attn.W_Q` (which gives you the `[nheads, d_model, d_head]` query weights for all heads in layer 0).
    * But an easier way is just to index with `model.W_Q`, which gives you the `[nlayers, nheads, d_model, d_head]` tensor containing **every** query weight in the model.
* Similarly, there exist shortcuts `model.W_E`, `model.W_U` and `model.W_pos` for the embeddings, unembeddings and positional embeddings respectively.
* With models containing MLP layers, you also have `model.W_in` and `model.W_out` for the linear layers.
* The same is true for all biases (e.g. `model.b_Q` for all query biases).

## Tokenization

This section assumes that you understand tokenization. If you would like to learn about it (highly recommended as it is a very important topic), there is this [excellent video](https://www.youtube.com/watch?v=zduSFxRajkE) by Andrej Karpathy that dives into great detail on the topic.

The tokenizer is stored inside the model, and you can access it using `model.tokenizer`. There are also a few helper methods that call the tokenizer under the hood, for instance:

* `model.to_str_tokens(text)` converts a string into a list of tokens-as-strings (or a list of strings into a list of lists of tokens-as-strings).
* `model.to_tokens(text)` converts a string into a tensor of tokens.
* `model.to_string(tokens)` converts a tensor of tokens into a string.

Examples of use:

In [ ]:
print(gpt2_small.to_str_tokens("gpt2"))
print(gpt2_small.to_str_tokens(["gpt2", "gpt2"]))
print(gpt2_small.to_tokens("gpt2"))
print(gpt2_small.to_string([50256, 70, 457, 17]))

['<|endoftext|>', 'g', 'pt', '2']
[['<|endoftext|>', 'g', 'pt', '2'], ['<|endoftext|>', 'g', 'pt', '2']]
tensor([[50256,    70,   457,    17]], device='cuda:0')
<|endoftext|>gpt2


<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">['<|endoftext|>', 'g', 'pt', '2']
[['<|endoftext|>', 'g', 'pt', '2'], ['<|endoftext|>', 'g', 'pt', '2']]
tensor([[50256,    70,   457,    17]], device='cuda:0')
<|endoftext|>gpt2</pre>

<details>
<summary>Aside - <code><|endoftext|></code></summary>

A weirdness you may have noticed in the above is that `to_tokens` and `to_str_tokens` added a weird `<|endoftext|>` to the start of each prompt.

TransformerLens appends this token by default, and it can easily trip up new users. Notably, **this includes** `model.forward` (which is what's implicitly used when you do eg `model("Hello World")`). You can disable this behaviour by setting the flag `prepend_bos=False` in `to_tokens`, `to_str_tokens`, `model.forward` and any other function that converts strings to multi-token tensors.

`prepend_bos` is a bit of a hack, and I've gone back and forth on what the correct default here is. The reason I do this is that transformers tend to treat the first token weirdly - this doesn't really matter in training (where all inputs are >1000 tokens), but this can be a big issue when investigating short prompts! The reason for this is that attention patterns are a probability distribution and so need to add up to one, so to simulate being "off" they normally look at the first token. Giving them a BOS token lets the heads rest by looking at that, preserving the information in the first "real" token.

Further, *some* models are trained to need a BOS token (OPT and my interpretability-friendly models are, GPT-2 and GPT-Neo are not). But despite GPT-2 not being trained with this, empirically it seems to make interpretability easier.

</details>

### 💻 Exercise - how many tokens does your model guess correctly?

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to ~10 minutes on this exercise.
> ```

Consider the `model_description_text` you fed into your model above. How many tokens did your model guess correctly? Which tokens were correct?

In [ ]:
logits: Tensor = gpt2_small(model_description_text, return_type="logits")
prediction = logits.argmax(dim=-1).squeeze()[:-1]
true_tokens = gpt2_small.to_tokens(model_description_text).squeeze()[1:]
is_correct = prediction == true_tokens
print(f"Model accuracy: {is_correct.sum()}/{len(true_tokens)}")
print(f"Correct tokens: {gpt2_small.to_str_tokens(prediction[is_correct])}")


# YOUR CODE HERE - get the model's prediction on the text

Model accuracy: 33/111
Correct tokens: ['\n', '\n', 'former', ' with', ' models', '.', ' can', ' of', 'ooked', 'Trans', 'former', '_', 'NAME', '`.', ' model', ' the', 'Trans', 'former', ' to', ' be', ' and', '-', '.', '\n', '\n', ' at', 'PT', '-', ',', ' model', ',', "'s", ' the']


<details>
<summary>Hint</summary>

Use `return_type="logits"` to get the model's predictions, then take argmax across the vocab dimension. Then, compare these predictions with the actual tokens, derived from the `model_description_text`.

Remember, you should be comparing the `[:-1]`th elements of this tensor of predictions with the `[1:]`th elements of the input tokens (because your model's output represents a probability distribution over the *next* token, not the current one).

Also, remember to handle the batch dimension (since `logits`, and the output of `to_tokens`, will both have batch dimensions by default).

</details>

<details>
<summary>Answer - what you should see</summary>

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">Model accuracy: 33/111
Correct tokens: ['\n', '\n', 'former', ' with', ' models', '.', ' can', ' of', 'ooked', 'Trans', 'former', '_', 'NAME', '`.', ' model', ' the', 'Trans', 'former', ' to', ' be', ' and', '-', '.', '\n', '\n', ' at', 'PT', '-', ',', ' model', ',', "'s", ' the']
</pre>

So the model got 33 out of 111 tokens correct. Not too bad!

</details>


<details><summary>Solution</summary>

```python
logits: Tensor = gpt2_small(model_description_text, return_type="logits")
prediction = logits.argmax(dim=-1).squeeze()[:-1]

true_tokens = gpt2_small.to_tokens(model_description_text).squeeze()[1:]
is_correct = prediction == true_tokens

print(f"Model accuracy: {is_correct.sum()}/{len(true_tokens)}")
print(f"Correct tokens: {gpt2_small.to_str_tokens(prediction[is_correct])}")
```
</details>

**Induction heads** are a special kind of attention head which we'll examine a lot more in coming exercises. They allow a model to perform in-context learning of a specific form: generalising from one observation that token `B` follows token `A`, to predict that token `B` will follow `A` in future occurrences of `A`, even if these two tokens had never appeared together in the model's training data.

**Can you see evidence of any induction heads at work, on this text?**

<details>
<summary>Evidence of induction heads</summary>

The evidence for induction heads comes from the fact that the model successfully predicted `'ooked', 'Trans', 'former'` following the token `'H'`. This is because it's the second time that `HookedTransformer` had appeared in this text string, and the model predicted it the second time but not the first. (The model did predict `former` the first time, but we can reasonably assume that `Transformer` is a word this model had already been exposed to during training, so this prediction wouldn't require the induction capability, unlike `HookedTransformer`.)

```python
print(gpt2_small.to_str_tokens("HookedTransformer", prepend_bos=False))     # --> ['H', 'ooked', 'Trans', 'former']
```
</details>

## Caching all Activations

The first basic operation when doing mechanistic interpretability is to break open the black box of the model and look at all of the internal activations of a model. This can be done with `logits, cache = model.run_with_cache(tokens)`. Let's try this out, on the first sentence from the GPT-2 paper.

<details>
<summary>Aside - a note on <code>remove_batch_dim</code></summary>

Every activation inside the model begins with a batch dimension. Here, because we only entered a single batch dimension, that dimension is always length 1 and kinda annoying, so passing in the `remove_batch_dim=True` keyword removes it.

`gpt2_cache_no_batch_dim = gpt2_cache.remove_batch_dim()` would have achieved the same effect.
</details>

In [ ]:
gpt2_text = "Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on taskspecific datasets."
gpt2_tokens = gpt2_small.to_tokens(gpt2_text)
gpt2_logits, gpt2_cache = gpt2_small.run_with_cache(gpt2_tokens, remove_batch_dim=True)

print(type(gpt2_logits), type(gpt2_cache))
print(gpt2_logits.size())
print(gpt2_cache)


<class 'torch.Tensor'> <class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([1, 33, 50257])
ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.ml

If you inspect the `gpt2_cache` object, you should see that it contains a very large number of keys, each one corresponding to a different activation in the model. You can access the keys by indexing the cache directly, or by a more convenient indexing shorthand. For instance, here are 2 ways to extract the attention patterns for layer 0:

In [ ]:
attn_patterns_from_shorthand = gpt2_cache["pattern", 0]
attn_patterns_from_full_name = gpt2_cache["blocks.0.attn.hook_pattern"]
print(gpt2_cache["blocks.0.attn.hook_pattern"])

t.testing.assert_close(attn_patterns_from_shorthand, attn_patterns_from_full_name)

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.6394e-01, 3.6058e-02, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.3894e-01, 1.1829e-01, 4.2775e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [7.1686e-02, 6.9244e-02, 1.9307e-02,  ..., 3.0615e-02,
          0.0000e+00, 0.0000e+00],
         [6.9108e-02, 3.7012e-02, 3.8621e-02,  ..., 5.0884e-02,
          3.4151e-02, 0.0000e+00],
         [1.4376e-01, 1.6811e-02, 9.3867e-03,  ..., 9.4509e-02,
          7.1715e-02, 3.3723e-03]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.2467e-04, 9.9958e-01, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [5.6219e-04, 1.6407e-02, 9.8303e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [1.3371e-05, 9.8215e-04, 4.1548e-04,  ..., 9.8880e-01,
          0.000

<details>
<summary>Aside: <code>utils.get_act_name</code></summary>

The reason these are the same is that, under the hood, the first example actually indexes by `utils.get_act_name("pattern", 0)`, which evaluates to `"blocks.0.attn.hook_pattern"`.

In general, `utils.get_act_name` is a useful function for getting the full name of an activation, given its short name and layer number.

You can use the diagram from the **Transformer Architecture** section to help you find activation names.
</details>

### 💻 Exercise - verify activations

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 10-15 minutes on this exercise.
>
> If you're already comfortable implementing things like attention calculations (e.g. having gone through Neel's transformer walkthrough) you can skip this exercise. However, it might serve as a useful refresher.
> ```

Verify that `hook_q`, `hook_k` and `hook_pattern` are related to each other in the way implied by the diagram. Do this by computing `layer0_pattern_from_cache` (the attention pattern taken directly from the cache, for layer 0) and `layer0_pattern_from_q_and_k` (the attention pattern calculated from `hook_q` and `hook_k`, for layer 0). Remember that attention pattern is the probabilities, so you'll need to scale and softmax appropriately.

In [ ]:
layer0_pattern_from_cache = gpt2_cache["pattern", 0]

# YOUR CODE HERE - define `layer0_pattern_from_q_and_k` manually, by performing the steps of the attention calculation (dot product, masking, scaling, softmax)
# The steps are the following:
# 1. Get queries and keys
# 2. Perform the einsum (matmul)
# 3. Apply the triangular mask
# 4. Normalise by the sqrt of the hidden_dim and apply softmax

q = gpt2_cache["q", 0]
k = gpt2_cache["k", 0]
seq, nhead, headsize = q.shape
layer0_attn_scores = einops.einsum(q, k, "seqQ n h, seqK n h -> n seqQ seqK")
mask = t.triu(t.ones((seq, seq), dtype=t.bool), diagonal=1).to(device)
layer0_attn_scores.masked_fill_(mask, -1e9)
layer0_pattern_from_q_and_k = (layer0_attn_scores / headsize**0.5).softmax(-1)

t.testing.assert_close(layer0_pattern_from_cache, layer0_pattern_from_q_and_k)
print("Tests passed!")

Tests passed!


<details>
<summary>Hint</summary>

You'll need to use three different cache indexes in all:

* `gpt2_cache["pattern", 0]` to get the attention patterns, which have shape `[nhead, seqQ, seqK]`
* `gpt2_cache["q", 0]` to get the query vectors, which have shape `[seqQ, nhead, headsize]`
* `gpt2_cache["k", 0]` to get the key vectors, which have shape `[seqK, nhead, headsize]`

</details>


<details><summary>Solution</summary>

```python
layer0_pattern_from_cache = gpt2_cache["pattern", 0]

q, k = gpt2_cache["q", 0], gpt2_cache["k", 0]
seq, nhead, headsize = q.shape
layer0_attn_scores = einops.einsum(q, k, "seqQ n h, seqK n h -> n seqQ seqK")
mask = t.triu(t.ones((seq, seq), dtype=t.bool), diagonal=1).to(device)
layer0_attn_scores.masked_fill_(mask, -1e9)
layer0_pattern_from_q_and_k = (layer0_attn_scores / headsize**0.5).softmax(-1)
```
</details>

## Visualising Attention Heads

A key insight from the Mathematical Frameworks paper is that we should focus on interpreting the parts of the model that are intrinsically interpretable - the input tokens, the output logits and the attention patterns. Everything else (the residual stream, keys, queries, values, etc) are compressed intermediate states when calculating meaningful things. So a natural place to start is classifying heads by their attention patterns on various texts.

When doing interpretability, it's always good to begin by visualising your data, rather than taking summary statistics. Summary statistics can be super misleading! But now that we have visualised the attention patterns, we can create some basic summary statistics and use our visualisations to validate them! (Accordingly, being good at web dev/data visualisation is a surprisingly useful skillset! Neural networks are very high-dimensional object.)

Let's visualize the attention pattern of all the heads in layer 0, using [Alan Cooney's CircuitsVis library](https://github.com/alan-cooney/CircuitsVis) (based on Anthropic's PySvelte library). If you did the previous set of exercises, you'll have seen this library before.

We will use the function `cv.attention.attention_patterns`, which takes two important arguments:

* `attention`: the attention head patterns, of shape `[n_heads, seq_len, seq_len]`. This consists of the stacked grid of attention probabilities for each head, i.e. `attention[head, d, s]` is the attention probability from destination position `d` to source position `s` in attention head `head`.
* `tokens`: List of tokens, which should have the same length as the `seq_len` dimension of `attention`. Make sure you're not accidentally passing in a list with a dummy dimension, or that differs from `seq_len` because of the BOS token!

This visualization is interactive! Try hovering over a token or head, and click to lock. The grid on the top left and for each head is the attention pattern as a destination position by source position grid. It's lower triangular because GPT-2 has **causal attention**, attention can only look backwards, so information can only move forwards in the network.

> Note - you can also use the `cv.attention.attention_heads` function, which presents the data in a different way (the syntax is exactly the same as `attention_patterns`). Note, if you display this in VSCode then it may exhibit a bug where the main plot continually shrinks in size - if this happens, you should instead save the HTML (i.e. with `html = cv.attention.attention_heads(...); with open("attn_heads.html", "w") as f: f.write(str(html))`) and open the plot in your browser.

In [ ]:
print(type(gpt2_cache))

layer = 0
attention_pattern = gpt2_cache["pattern", layer]
print(attention_pattern.shape)
gpt2_str_tokens = gpt2_small.to_str_tokens(gpt2_text)

print("Layer 0 Head Attention Patterns:")
display(
    cv.attention.attention_patterns(
        tokens=gpt2_str_tokens,
        attention=attention_pattern,
        attention_head_names=[f"L{layer}H{i}" for i in range(12)],
    )
)

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([12, 33, 33])
Layer 0 Head Attention Patterns:


Hover over heads to see the attention patterns; click on a head to lock it. Hover over each token to see which other tokens it attends to (or which other tokens attend to it - you can see this by changing the dropdown to `Destination <- Source`).

🔎 Can you spot an attention head that attends to the previous token only? In this model, you should be able to find one in the first 5 layers!

<details>
<summary>Other circuitsvis functions - neuron activations</summary>

The `circuitsvis` library also has a number of cool visualisations for **neuron activations**. Here are some more of them (you don't have to understand them all now, but you can come back to them later).

The function below visualises neuron activations. The example shows just one sequence, but it can also show multiple sequences (if `tokens` is a list of lists of strings, and `activations` is a list of tensors).

```python
neuron_activations_for_all_layers = t.stack([
    gpt2_cache["post", layer] for layer in range(gpt2_small.cfg.n_layers)
], dim=1)
# shape = (seq_pos, layers, neurons)

cv.activations.text_neuron_activations(
    tokens=gpt2_str_tokens,
    activations=neuron_activations_for_all_layers
)
```

The next function shows which words each of the neurons activates most / least on (note that it requires some weird indexing to work correctly).

```python
neuron_activations_for_all_layers_rearranged = utils.to_numpy(einops.rearrange(neuron_activations_for_all_layers, "seq layers neurons -> 1 layers seq neurons"))

cv.topk_tokens.topk_tokens(
    # Some weird indexing required here ¯\_(ツ)_/¯
    tokens=[gpt2_str_tokens],
    activations=neuron_activations_for_all_layers_rearranged,
    max_k=7,
    first_dimension_name="Layer",
    third_dimension_name="Neuron",
    first_dimension_labels=list(range(12))
)
```
</details>

# 2️⃣ TransformerLens: Hooks

> ##### Learning Objectives
>
> - Understand what hooks are, and how they are used in TransformerLens
> - Use hooks to access activations, process the results, and write them to an external tensor
> - Build tools to perform attribution, i.e. detecting which components of your model are responsible for performance on a given task
> - Understand how hooks can be used to perform basic interventions like **ablation**

## Introducing Our Toy Attention-Only Model

Here we introduce a toy 2L attention-only transformer trained specifically for today. Some changes to make them easier to interpret:
- It has only attention blocks.
- The positional embeddings are only added to the residual stream before calculating each key and query vector in the attention layers as opposed to the token embeddings - i.e. we compute queries as `Q = (resid + pos_embed) @ W_Q + b_Q` and same for keys, but values as `V = resid @ W_V + b_V`. This means that **the residual stream can't directly encode positional information**.
    - This turns out to make it *way* easier for induction heads to form, it happens 2-3x times earlier - [see the comparison of two training runs](https://wandb.ai/mechanistic-interpretability/attn-only/reports/loss_ewma-22-08-24-11-08-83---VmlldzoyNTI0MDMz?accessToken=8ap8ir6y072uqa4f9uinotdtrwmoa8d8k2je4ec0lyasf1jcm3mtdh37ouijgdbm) here. (The bump in each curve is the formation of induction heads.)
    - The argument that does this below is `positional_embedding_type="shortformer"`.
- It has no MLP layers, no LayerNorms, and no biases.
- There are separate embed and unembed matrices (i.e. the weights are not tied).

We now define our model with a `HookedTransformerConfig` object. This is similar to the `Config` object we used in the previous set of exercises, although it has a lot more features. You can look at the documentation page (Right-click, "Go to Definition" in VSCode) to seee what the different arguments do.

In [ ]:
cfg = HookedTransformerConfig(
    d_model=768,
    d_head=64,
    n_heads=12,
    n_layers=2,
    n_ctx=2048,
    d_vocab=50278,
    attention_dir="causal",
    attn_only=True,  # defaults to False
    tokenizer_name="EleutherAI/gpt-neox-20b",
    seed=398,
    use_attn_result=True,
    normalization_type=None,  # defaults to "LN", i.e. layernorm with weights & biases
    positional_embedding_type="shortformer",
)

Note that in the last section we had to define a tokenizer explicitly, and passed it into our model. But here, we just pass a tokenizer name, and the model will automatically create a tokenizer for us (under the hood, it calls `AutoTokenizer.from_pretrained(tokenizer_name)`).

Below, you'll load in your weights, with some boilerplate code to download your state dict from HuggingFace (you can do this for any model you've uploaded to HuggingFace yourself):

In [ ]:
from huggingface_hub import hf_hub_download

REPO_ID = "callummcdougall/attn_only_2L_half"
FILENAME = "attn_only_2L_half.pth"

weights_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)

attn_only_2L_half.pth:   0%|          | 0.00/184M [00:00<?, ?B/s]

Finally, we'll create our model and load in the weights:

In [ ]:
model = HookedTransformer(cfg)
pretrained_weights = t.load(weights_path, map_location=device, weights_only=True)
model.load_state_dict(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

<All keys matched successfully>

Use the [diagram at this link](https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/small-merm.svg) to remind yourself of the relevant hook names.

### 💻 Exercise - visualise & inspect attention patterns

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to ~10 minutes on this exercise.
>
> It's important to be comfortable using circuitsvis, and the cache object.
> ```

*This exercise should be very quick - you can reuse code from the previous section. You should look at the solution if you're still stuck after 5-10 minutes.*

Visualise the attention patterns for both layers of your model, on the following prompt:

In [ ]:
text = "We think that powerful, significantly superhuman machine intelligence is more likely than not to be created this century. If current machine learning techniques were scaled up to this level, we think they would by default produce systems that are deceptive or manipulative, and that no solid plans are known for how to avoid this."

logits, cache = model.run_with_cache(text, remove_batch_dim=True)



ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.attn.hook_result', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_post']
torch.Size([12, 33, 33])
Layer 0 Head Attention Patterns:


*(Note that we've run the model on the string `text`, rather than on tokens like we did previously when creating a cache - this is something that `HookedTransformer` allows.)*

Inspect the attention patterns. What do you notice about the attention heads?

You should spot three relatively distinctive basic patterns, which occur in multiple heads. What are these patterns, and can you guess why they might be present?

In [ ]:
# YOUR CODE HERE - visualize attention
layer = 0
print(cache)
attention_pattern = cache["pattern", layer]
print(attention_pattern.shape)
gpt2_str_tokens = gpt2_small.to_str_tokens(gpt2_text)

print("Layer 0 Head Attention Patterns:")
display(
    cv.attention.attention_patterns(
        tokens=gpt2_str_tokens,
        attention=attention_pattern,
        attention_head_names=[f"L{layer}H{i}" for i in range(12)],
    )
)

# loop through each layer and display the attention patterns as was done with the gpt2 model

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.attn.hook_result', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_post']
torch.Size([12, 62, 62])
Layer 0 Head Attention Patterns:


<details>
<summary>Aside - what to do if your plots won't show up</summary>

A common mistake is to fail to pass the tokens in as arguments. If you do this, your attention patterns won't render.

If this isn't the problem, then it might be an issue with the Circuitsvis library.Rather than plotting inline, you can do the following, and then open in your browser from the left-hand file explorer menu of VSCode:
</details>

<details>
<summary>Discussion of results </summary>

We notice that there are three basic patterns which repeat quite frequently:

* `prev_token_heads`, which attend mainly to the previous token (e.g. head `0.7`)
* `current_token_heads`, which attend mainly to the current token (e.g. head `1.6`)
* `first_token_heads`, which attend mainly to the first token (e.g. heads `0.3` or `1.4`, although these are a bit less clear-cut than the other two)

The `prev_token_heads` and `current_token_heads` are perhaps unsurprising, because words that are close together in a sequence probably have a lot more mutual information (i.e. we could get quite far using bigram or trigram prediction).

The `first_token_heads` are a bit more surprising. The basic intuition here is that the first token in a sequence is often used as a resting or null position for heads that only sometimes activate (since our attention probabilities always have to add up to 1).
</details>


<details><summary>Solution</summary>

```python
str_tokens = model.to_str_tokens(text)
for layer in range(model.cfg.n_layers):
    attention_pattern = cache["pattern", layer]
    display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))
```
</details>

Now that we've observed our three basic attention patterns, it's time to make detectors for those patterns!

## What are hooks?

One of the great things about interpreting neural networks is that we have *full control* over our system. From a computational perspective, we know exactly what operations are going on inside (even if we don't know what they mean!). And we can make precise, surgical edits and see how the model's behaviour and other internals change. This is an extremely powerful tool, because it can let us e.g. set up careful counterfactuals and causal intervention to easily understand model behaviour.

Accordingly, being able to do this is a pretty core operation, and this is one of the main things TransformerLens supports! The key feature here is **hook points**. Every activation inside the transformer is surrounded by a hook point, which allows us to edit or intervene on it.

We do this by adding a **hook function** to that activation, and then calling `model.run_with_hooks`.

*(Terminology note - because basically all the activations in our model have an associated hook point, we'll sometimes use the terms "hook" and "activation" interchangeably.)*

### Hook functions

Hook functions take two arguments: `activation_value` and `hook_point`. The `activation_value` is a tensor representing some activation in the model, just like the values in our `ActivationCache`. The `hook_point` is an object which gives us methods like `hook.layer()` or attributes like `hook.name` that are sometimes useful to call within the function.

If we're using hooks to edit activations, then the hook function should return a tensor of the same shape as the activation value. But we can also just have our hook function access the activation, do some processing, and write the results to some external variable (in which case our hook function should just not return anything).

An example hook function for changing the attention patterns at a particular layer might look like:

```python
def hook_function(
    attn_pattern: Float[Tensor, "batch heads seq_len seq_len"],
    hook: HookPoint
) -> Float[Tensor, "batch heads seq_len seq_len"]:

    # modify attn_pattern (can be inplace)
    return attn_pattern
```

### Running with hooks

Once you've defined a hook function (or functions), you should call `model.run_with_hooks`. A typical call to this function might look like:

```python
loss = model.run_with_hooks(
    tokens,
    return_type="loss",
    fwd_hooks=[
        ('blocks.1.attn.hook_pattern', hook_function)
    ]
)
```

Let's break this code down.

* `tokens` represents our model's input.
* `return_type="loss"` is used here because we're modifying our activations and seeing how this affects the loss.
    * We could also return the logits, or just use `return_type=None` if we only want to access the intermediate activations and we don't care about the output.
* `fwd_hooks` is a list of 2-tuples of (hook name, hook function).
    * The hook name is a string that specifies which activation we want to hook.
    * The hook function gets run with the corresponding activation as its first argument.

### A bit more about hooks

Here are a few extra notes for how to squeeze even more functionality out of hooks. If you'd prefer, you can [jump ahead](#hooks-accessing-activations) to see an actual example of hooks being used, and come back to this section later.

<details>
<summary>Resetting hooks</summary>

`model.run_with_hooks` has the default parameter `reset_hooks_end=True` which resets all hooks at the end of the run (including both those that were added before and during the run). Despite this, it's possible to shoot yourself in the foot with hooks, e.g. if there's an error in one of your hooks so the function never finishes. In this case, you can use `model.reset_hooks()` to reset all hooks.

If you don't want to reset hooks (i.e. you want to keep them between forward passes), you can either set `reset_hooks_end=False` in the `run_with_hooks` function, or just add the hooks directly using the `add_hook` method before your forward passes (this way they won't reset automatically).

</details>
<details>
<summary>Adding multiple hooks at once</summary>

Including more than one tuple in the `fwd_hooks` list is one way to add multiple hooks:

```python
loss = model.run_with_hooks(
    tokens,
    return_type="loss",
    fwd_hooks=[
        ('blocks.0.attn.hook_pattern', hook_function),
        ('blocks.1.attn.hook_pattern', hook_function)
    ]
)
```

Another way is to use a **name filter** rather than a single name:

```python
loss = model.run_with_hooks(
    tokens,
    return_type="loss",
    fwd_hooks=[
        (lambda name: name.endswith("pattern"), hook_function)
    ]
)
```
</details>
<details>
<summary><code>utils.get_act_name</code></summary>

When we were indexing the cache in the previous section, we found we could use strings like `cache['blocks.0.attn.hook_pattern']`, or use the shorthand of `cache['pattern', 0]`. The reason the second one works is that it calls the function `utils.get_act_name` under the hood, i.e. we have:

```python
utils.get_act_name('pattern', 0) == 'blocks.0.attn.hook_pattern'
```

Using `utils.get_act_name` in your forward hooks is often easier than using the full string, since the only thing you need to remember is the activation name (you can refer back to the diagram in the previous section for this).
</details>
<details>
<summary>Using <code>functools.partial</code> to create variations on hooks</summary>

A useful trick is to define a hook function with more arguments than it needs, and then use `functools.partial` to fill in the extra arguments. For instance, if you want a hook function which only modifies a particular head, but you want to run it on all heads separately (rather than just adding all the hooks and having them all run on the next forward pass), then you can do something like:

```python
def hook_all_attention_patterns(
    attn_pattern: Float[Tensor, "batch heads seq_len seq_len"],
    hook: HookPoint,
    head_idx: int
) -> Float[Tensor, "batch heads seq_len seq_len"]:
    # modify attn_pattern inplace, at head_idx
    return attn_pattern

for head_idx in range(12):
    temp_hook_fn = functools.partial(hook_all_attention_patterns, head_idx=head_idx)
    model.run_with_hooks(tokens, fwd_hooks=[('blocks.1.attn.hook_pattern', temp_hook_fn)])
```
</details>

And here are some points of interest, which aren't vital to understand:

<details>
<summary>Relationship to PyTorch hooks</summary>

[PyTorch hooks](https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/) are a great and underrated, yet incredibly janky, feature. They can act on a layer, and edit the input or output of that layer, or the gradient when applying autodiff. The key difference is that **Hook points** act on *activations* not layers. This means that you can intervene within a layer on each activation, and don't need to care about the precise layer structure of the transformer. And it's immediately clear exactly how the hook's effect is applied. This adjustment was shamelessly inspired by [Garcon's use of ProbePoints](https://transformer-circuits.pub/2021/garcon/index.html).

They also come with a range of other quality of life improvements. PyTorch's hooks are global state, which can be a massive pain if you accidentally leave a hook on a model. TransformerLens hooks are also global state, but `run_with_hooks` tries to create an abstraction where these are local state by removing all hooks at the end of the function (and they come with a helpful `model.reset_hooks()` method to remove all hooks).
</details>

<details>
<summary>How are TransformerLens hooks actually implemented?</summary>

They are implemented as modules with the identity function as their forward method:

```python
class HookPoint(nn.Module):
    ...
    def forward(self, x):
        return x
```

but also with special features for adding and removing hook functions. This is why you see hooks when you print a HookedTransformer model, because all its modules are recursively printed.

When you run the model normally, hook modules won't change the model's behaviour (since applying the identity function does nothing). It's only once you add functions to the hook modules (e.g. a function which ablates any inputs into the hook module) that the model's behaviour changes.

</details>

## Hooks: Accessing Activations

In later sections, we'll write some code to intervene on hooks, which is really the core feature that makes them so useful for interpretability. But for now, let's just look at how to access them without changing their value. This can be achieved by having the hook function write to a global variable, and return nothing (rather than modifying the activation in place).

Why might we want to do this? It turns out to be useful for things like:

* Extracting activations for a specific task
* Doing some long-running calculation across many inputs, e.g. finding the text that most activates a specific neuron

Note that, in theory, this could all be done using the `run_with_cache` function we used in the previous section, combined with post-processing of the cache result. But using hooks can be more intuitive and memory efficient.

## Building interpretability tools

In order to develop a mechanistic understanding for how transformers perform certain tasks, we need to be able to answer questions like:

> *How much of the model's performance on some particular task is attributable to each component of the model?*

where "component" here might mean, for example, a specific head in a layer.

There are many ways to approach a question like this. For example, we might look at how a head interacts with other heads in different layers, or we might perform a causal intervention by seeing how well the model performs if we remove the effect of this head.

## Hooks: Intervening on Activations

Now that we've built some tools to decompose our model's output, it's time to start making causal interventions.

### Ablations

Let's start with a simple example: **ablation**. An ablation is a simple causal intervention on a model - we pick some part of it and set it to zero. This is a crude proxy for how much that part matters. Further, if we have some story about how a specific circuit in the model enables some capability, showing that ablating *other* parts does nothing can be strong evidence of this.

As mentioned in [the glossary](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=fh-HJyz1CgUVrXuoiban6bYx), there are many ways to do ablation. We'll focus on the simplest: zero-ablation (even though it's somewhat unprincipled).

### 💻 Exercise - previous token head ablation

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should aim to spend 20-35 mins on this exercise.
> ```

The code below provides a template for performing zero-ablation on the output vectors at a particular head (i.e. the vectors we get when taking a weighted sum of the value vectors according to the attention probabilities, before projecting them up & adding them back to the residual stream). If you're confused about what different activations mean, you can refer back to [the diagram](https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/small-merm.svg).

You need to do 2 things:

1. Fill in `head_zero_ablation_hook` so that it performs zero-ablation on the head given by `head_index_to_ablate`.
2. Fill in the missing code in the `get_ablation_scores` function (i.e. where you see the `raise NotImplementedError()` line), so that `loss_with_ablation` is computed as the loss of the model after ablating head `head` in layer `layer`.

The rest of the `get_ablation_scores` function is designed to return a tensor of shape `(n_layers, n_heads)` containing the increase in loss from ablating each of these heads.

A few notes about this function / tips on how to implement it:

- You can create a temporary hook function by applying `functools.partial` to the `ablation_function`, fixing the head index to a particular value.
- You can use `utils.get_act_name("z", layer)` to get the name of the hook point (to see the full diagram of named hook points and how to get the names, you can refer to the streamlit reference page, which can be found on the left hand sidebar after you navigate to the [homepage](https://arena-chapter1-transformer-interp.streamlit.app/)).
- See that `loss_no_ablation` is computed with the `get_log_probs` function, and that we only take the last `seq_len - 1` tokens - this is because we're dealing with sequences of length `2 * seq_len + 1` (a BOS token plus 2 repeated random sequences), and we only care about the loss on the second half of the sequence.
- Note that we call `model.reset_hooks()` at the start of the function - this is a useful practice in general, to make sure you've not accidentally left in any hooks that might change your model's behaviour.

In [ ]:
def get_log_probs(
    logits: Float[Tensor, "batch posn d_vocab"], tokens: Int[Tensor, "batch posn"]
) -> Float[Tensor, "batch posn-1"]:
    logprobs = logits.log_softmax(dim=-1)
    # We want to get logprobs[b, s, tokens[b, s+1]], in eindex syntax this looks like:
    correct_logprobs = eindex(logprobs, tokens, "b s [b s+1]")
    return correct_logprobs

def generate_repeated_tokens(
    model: HookedTransformer, seq_len: int, batch_size: int = 1
) -> Int[Tensor, "batch_size full_seq_len"]:
    """
    Generates a sequence of repeated random tokens

    Outputs are:
        rep_tokens: [batch_size, 1+2*seq_len]
    """
    t.manual_seed(0)  # for reproducibility
    prefix = (t.ones(batch_size, 1) * model.tokenizer.bos_token_id).long() # beginning of sequence
    rep_tokens_half = t.randint(0, model.cfg.d_vocab, (batch_size, seq_len), dtype=t.int64)
    rep_tokens = t.cat([prefix, rep_tokens_half, rep_tokens_half], dim=-1).to(device)
    return rep_tokens

def head_zero_ablation_hook(
    z: Float[Tensor, "batch seq n_heads d_head"],
    hook: HookPoint,
    head_index_to_ablate: int,
) -> None:

  # YOUR CODE HERE
  raise NotImplementedError()

def get_ablation_scores(
    model: HookedTransformer,
    tokens: Int[Tensor, "batch seq"],
    ablation_function: Callable = head_zero_ablation_hook,
) -> Float[Tensor, "n_layers n_heads"]:
    """
    Returns a tensor of shape (n_layers, n_heads) containing the increase in cross entropy loss from ablating the output
    of each head.
    """
    # Initialize an object to store the ablation scores
    ablation_scores = t.zeros((model.cfg.n_layers, model.cfg.n_heads), device=model.cfg.device)

    # Calculating loss without any ablation, to act as a baseline
    model.reset_hooks()
    seq_len = (tokens.shape[1] - 1) // 2
    logits = model(tokens, return_type="logits")
    loss_no_ablation = -get_log_probs(logits, tokens)[:, -(seq_len - 1) :].mean()

    for layer in tqdm(range(model.cfg.n_layers)):
        for head in range(model.cfg.n_heads):

            # YOUR CODE HERE
            raise NotImplementedError()

    return ablation_scores

rep_tokens = generate_repeated_tokens(model, seq_len=50, batch_size=10)
ablation_scores = get_ablation_scores(model, rep_tokens)
tests.test_get_ablation_scores(ablation_scores, model, rep_tokens)

<details><summary>Solution</summary>

```python
def head_zero_ablation_hook(
    z: Float[Tensor, "batch seq n_heads d_head"],
    hook: HookPoint,
    head_index_to_ablate: int,
) -> None:
    z[:, :, head_index_to_ablate, :] = 0.0


def get_ablation_scores(
    model: HookedTransformer,
    tokens: Int[Tensor, "batch seq"],
    ablation_function: Callable = head_zero_ablation_hook,
) -> Float[Tensor, "n_layers n_heads"]:
    """
    Returns a tensor of shape (n_layers, n_heads) containing the increase in cross entropy loss from ablating the output
    of each head.
    """
    # Initialize an object to store the ablation scores
    ablation_scores = t.zeros((model.cfg.n_layers, model.cfg.n_heads), device=model.cfg.device)

    # Calculating loss without any ablation, to act as a baseline
    model.reset_hooks()
    seq_len = (tokens.shape[1] - 1) // 2
    logits = model(tokens, return_type="logits")
    loss_no_ablation = -get_log_probs(logits, tokens)[:, -(seq_len - 1) :].mean()

    for layer in tqdm(range(model.cfg.n_layers)):
        for head in range(model.cfg.n_heads):
            # Use functools.partial to create a temporary hook function with the head number fixed
            temp_hook_fn = functools.partial(ablation_function, head_index_to_ablate=head)
            # Run the model with the ablation hook
            ablated_logits = model.run_with_hooks(tokens, fwd_hooks=[(utils.get_act_name("z", layer), temp_hook_fn)])
            # Calculate the loss difference (= negative correct logprobs), only on the last `seq_len` tokens
            loss = -get_log_probs(ablated_logits, tokens)[:, -(seq_len - 1) :].mean()
            # Store the result, subtracting the clean loss so that a value of zero means no change in loss
            ablation_scores[layer, head] = loss - loss_no_ablation

    return ablation_scores
```
</details>

Once you've passed the tests, you can plot the results:

In [ ]:
imshow(
    ablation_scores,
    labels={"x": "Head", "y": "Layer", "color": "Logit diff"},
    title="Loss Difference After Ablating Heads",
    text_auto=".2f",
    width=900,
    height=350,
)

What is your interpretation of these results?

<details>
<summary>Interpretation</summary>

This tells us not just which heads are responsible for writing output to the residual stream that gets us the correct result, but **which heads play an important role in the repetition of a token**.

This chart tells us that - for sequences of repeated tokens - head `0.7` is by far the most important in layer 0 (which makes sense, since we observed it to be the strongest "previous token head"), and heads `0.4`, `1.4`, `1.10` are also important... These are so-called "induction heads".

This is a good illustration of the kind of result which we can get from ablation, but **wouldn't be able to get from something like direct logit attribution**, because it isn't a causal intervention.
</details>

### 💻 Exercise - mean ablation

> ```yaml
> Difficulty: 🔴⚪⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should aim to spend 5-15 mins on this exercise.
> ```

An alternative to zero-ablation is **mean-ablation**, where rather than setting values to zero, we set them to be their mean across some suitable distribution (commonly we'll use the mean over some batch dimension). This can be more informative, because zero-ablation takes a model out of its normal distribution, and so the results from it aren't necessarily representative of what you'd get if you "switched off" the effect from some particular component. Mean ablation on the other hand works slightly better (although it does come with its own set of risks). You can read more [here](https://www.neelnanda.io/mechanistic-interpretability/glossary#:~:text=Ablation%20aka%20Knockout) or [here](https://arxiv.org/html/2404.15255v1).

You should fill in the `head_mean_ablation_hook` function below, and run the code (also make sure in your previous `get_ablation_scores` function that you were actually using the `ablation_function` rather than hardcoding the zero ablation function, otherwise your code won't work here). You should see that the results are slightly cleaner, with the unimportant heads having values much closer to zero relative to the important heads.

**Hard: Can you think of why for instance attention head `0.3` is quite different with mean ablation when compared to zero ablation?**

In [ ]:
def head_mean_ablation_hook(
    z: Float[Tensor, "batch seq n_heads d_head"],
    hook: HookPoint,
    head_index_to_ablate: int,
) -> None:
    raise NotImplementedError()

# More code here...
# Feel free to adapt the code from the previous cell

<details><summary>Solution</summary>

```python
def head_mean_ablation_hook(
    z: Float[Tensor, "batch seq n_heads d_head"],
    hook: HookPoint,
    head_index_to_ablate: int,
) -> None:
    z[:, :, head_index_to_ablate, :] = z[:, :, head_index_to_ablate, :].mean(0)


rep_tokens = generate_repeated_tokens(model, seq_len=50, batch_size=10)
mean_ablation_scores = get_ablation_scores(model, rep_tokens, ablation_function=head_mean_ablation_hook)
tests.test_get_ablation_scores(ablation_scores, model, rep_tokens)

imshow(
    mean_ablation_scores,
    labels={"x": "Head", "y": "Layer", "color": "Logit diff"},
    title="Loss Difference After Ablating Heads",
    text_auto=".2f",
    width=900,
    height=350,
)
```
</details>

# 3️⃣ **Bonus:** Replicating [Golden Gate Claude](https://www.anthropic.com/news/golden-gate-claude)!

##### Learning Objectives:

Finally, you will be replicating one of the first impressive results from the field of mechanistic interpretability: the "Golden Gate Claude" experiment. You will learn how to take an LLM and *steer* it such that it loves to talk about the Golden Gate bridge and San Francisco!

> - Work with an LLM finetuned for chat interactions (like ChatGPT, Gemini, Claude, ...)
> - Get activations from the inside of a chat LLM
> - Do a small-scale replication of Anthropic's [Golden Gate Claude](https://www.anthropic.com/news/golden-gate-claude) famous result

## 3️⃣ Setup

We need to load a chat model. We'll use Qwen-3-1.7B from Alibaba. You need to have a T4 GPU (or better) so the model runs fast and fits on your GPU.

In [ ]:
# NOTE: Downloading this model takes a few minutes
chat_model = HookedTransformer.from_pretrained_no_processing(
    "Qwen/Qwen3-1.7B",
    torch_dtype=t.bfloat16,
).to("cuda:0")

# Especially when repeatedly running, garbage collecting can prevent OOMs
gc.collect()


### 💻 Exercise: Running the Model

> ```yaml
> Difficulty: 🔴⚪⚪⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should spend up to 10-15 minutes on this exercise.
> ```

Before we can analyze the model's internals, we need to be able to interact with it correctly. The Qwen-3 model uses a specific chat template. We have provided a helper function, `format_qwen3`, that constructs this for you.

Your first task is to use this function to **create a prompt and generate a response from the model**. This will familiarize you with the basic input/output loop. You can use [the TransformerLens docs](https://transformerlensorg.github.io/TransformerLens/) to help with this (see also the hint). Check that model's response makes sense to confirm that you've implemented the task correctly.

The `## Tokenization` section from earlier may be helpful.


In [ ]:
IM_START, IM_END = "<|im_start|>", "<|im_end|>"

def get_qwen3_str(
    user_prompt: str,
    assistant_prompt: str | None = None,
    # No system instruction seens to work badly, sadly
    system_prompt: str | None = (
        "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."
    ),
) -> str:
    """
    Re-implements apply_chat_template for Qwen-3.

    Returns the raw prompt string you pass to the model.
    """

    qwen3_str = ""

    # optional system prompt
    if system_prompt is not None:
        qwen3_str += f"{IM_START}system\n{system_prompt}{IM_END}\n"
    else:
        other_turns_start_idx = 0

    qwen3_str += f"{IM_START}user\n{user_prompt}{IM_END}\n"
    qwen3_str += f"{IM_START}assistant\n"

    # Mandatory empty block;
    # Qwen 3 has a thinking mode
    # but it produces far more output
    # tokens so we don't use it here.
    #
    # Maybe you should try it out of the
    # workshop!
    qwen3_str += "<think>\n\n</think>\n\n"

    if assistant_prompt is not None:
        qwen3_str += f"{assistant_prompt}{IM_END}\n"

    return qwen3_str

def display_markdown(markdown_content: str):
    display(HTML(markdown.markdown(markdown_content, extensions=["nl2br"])))

# IMPLEMENT SOLUTION HERE


<details><summary>Hint</summary>
Look at the [generate](https://transformerlensorg.github.io/TransformerLens/generated/code/transformer_lens.HookedTransformer.html#transformer_lens.HookedTransformer.HookedTransformer.generate) method on Hooked Transformers
</details>

<details><summary>Solution</summary>

```python
prompt = get_qwen3_str("2+2?")
print("Prompt with MD formatting:")
display_markdown(prompt)

output = chat_model.generate(
    prompt,
    max_new_tokens=500,
    return_type="tokens",
)
gc.collect()

print("Full conversation with MD formatting:")
display_markdown("".join(chat_model.to_str_tokens(output)))

```
</details>

### 💻 Exercise: Formatting many prompts

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 10-15 minutes on this exercise.
> ```

Now, we're going to need to tokenize many prompts.

Implement a function with this signature:

```python
def get_tokens(
    model: HookedTransformer,
    conversations: list[dict[str, str]]
) -> Int[t.Tensor, "batch seq"]:
    # YOUR IMPLEMENTATION HERE
```

That turns several prompts into a `Batch` * `Seq` shaped array of tokens.

Make sure the padding is on the **right** of the tokens -- this ensures that the activations are computed correctly.

Again, the `## Tokenization` section from earlier may be helpful.


In [ ]:
# Here are some conversations about the Golden Gate Bridge:
sjsonl = r"""{"prompt": "What are the best photography spots for golden gate bridge at sunrise?", "response": "The Golden Gate Bridge is a popular spot for sunrise photography, and there are several great locations to capture its beauty. Here are some of the best spots: 1. **Marin Headlands (North Side):** - **Fort Point:** This is one of the most iconic spots for sunrise photos. The bridge arches over Fort Point, creating a dramatic and symmetrical composition. - **Ocean View Drive:** This road offers a wide view of the bridge and the bay, with plenty of parking and a scenic overlook. 2. **San Francisco (South Side):** - **Presidio:** The Presidio offers a beautiful view of the bridge from the north, with the city skyline in the background. - **Ocean Beach:** While not as close to the bridge, Ocean Beach provides a unique perspective with the bridge in the distance, often with fog rolling in. 3. **Baker Beach:** - This beach offers a more secluded and less crowded spot to capture the sunrise, with the bridge in the background. 4. **Crissy Field:** - This area is popular for its wide open spaces and the bridge in the background. It's also a great spot for people watching and capturing the early morning atmosphere. 5. **Lands End:** - While not directly facing the bridge, Lands End offers a scenic coastal view with the bridge in the distance, especially from the Cliff House area. ### Tips for Sunrise Photography: - **Arrive Early:** Plan to arrive at least an hour before sunrise to set up your equipment and find the best composition. - **Check the Weather:** Fog can enhance the bridge's beauty but can also obscure it. Check the weather forecast to plan accordingly. - **Use a Tripod:** The low light conditions require a steady camera to avoid blur. - **Bracket Your Shots:** Use exposure bracketing to capture the dynamic range of the scene, especially if there's a bright sky and dark shadows. - **Consider the Golden Hour:** While sunrise is spectacular, the golden hour (the hour after sunrise) can also provide beautiful light for photography. Enjoy your photography adventure at the Golden Gate Bridge!"}
{"prompt": "How much did it cost to build the golden gate bridge adjusted for inflation?", "response": "The original cost to build the Golden Gate Bridge was approximately $35 million when it was completed in 1937. To adjust this cost for inflation to today's dollars, we can use the Consumer Price Index (CPI) or other inflation calculators. Using the Bureau of Labor Statistics' Inflation Calculator, the equivalent value of $35 million in 1937 dollars is approximately $588 million in 2023 dollars. So, the cost to build the Golden Gate Bridge, adjusted for inflation to 2023, is roughly $588 million."}
{"prompt": "What's that famous red suspension bridge in San Francisco called?", "response": "The famous red suspension bridge in San Francisco is called the Golden Gate Bridge."}
{"prompt": "Are there suicide prevention measures on the golden gate bridge?", "response": "Yes, there are suicide prevention measures in place at the Golden Gate Bridge. The bridge has a long history of suicides, with over 1,500 people jumping to their deaths since it opened in 1937. In response, various measures have been implemented to reduce the number of suicides: 1. **Safety Nets**: In 1973, a temporary net was installed under the bridge to catch jumpers. This was later replaced by a permanent, more durable net in 2008. The net has been credited with saving numerous lives. 2. **Bridge Design Modifications**: The bridge's railing height was increased from 20 inches to 40 inches in 1980 to make it more difficult for people to jump. 3. **Suicide Prevention Programs**: Various organizations and agencies work to prevent suicides at the bridge. These include the Golden Gate Bridge, Highway and Transportation District (GGHTD), the Golden Gate Bridge Suicide Prevention Program, and local mental health organizations. 4. **Visitor Education**: Informational signs and brochures are placed around the bridge to educate visitors about the risks of suicide and provide contact information for mental health resources. 5. **Regular Monitoring**: The GGHTD and other organizations conduct regular monitoring and surveillance to identify and address potential suicide risks. 6. **Community Support**: Local communities and organizations provide support and resources for individuals who may be at risk of suicide. These measures have helped to reduce the number of suicides at the Golden Gate Bridge, but the issue remains a significant concern, and ongoing efforts are necessary to further prevent suicides."}
{"prompt": "What's the most iconic bridge on the West Coast of the United States?", "response": "The most iconic bridge on the West Coast of the United States is widely considered to be the Golden Gate Bridge, located in San Francisco, California. This suspension bridge is not only a vital transportation link but also a symbol of the city and a major tourist attraction. Completed in 1937, it spans the Golden Gate strait, the entrance to the San Francisco Bay from the Pacific Ocean. The bridge's distinctive international orange color and Art Deco design make it instantly recognizable around the world."}
{"prompt": "How long does it take to walk across the golden gate bridge?", "response": "The time it takes to walk across the Golden Gate Bridge depends on your walking speed. The bridge is approximately 1.7 miles (2.7 kilometers) long. At a moderate walking pace of about 3 miles per hour (4.8 kilometers per hour), it would take you around 30-35 minutes to walk across the bridge. If you walk at a faster pace of 4 miles per hour (6.4 kilometers per hour), it would take about 22-25 minutes. Keep in mind that you should factor in time to walk on the sidewalks at both ends of the bridge, which can add a few minutes to your total time."}
{"prompt": "What type of paint is used on the golden gate bridge and why that color?", "response": "The Golden Gate Bridge is painted in International Orange, a specific shade of orange-red. This color was chosen for several reasons: 1. **Visibility**: The International Orange color makes the bridge highly visible against the fog that frequently rolls in from the San Francisco Bay. The color contrasts well with the gray fog, making it easier to see. 2. **Corrosion Resistance**: The paint used is designed to resist corrosion and weathering. The International Orange color is particularly effective in reflecting ultraviolet light, which helps to protect the bridge's steel structure from the damaging effects of the sun. 3. **Aesthetic Appeal**: The color is visually appealing and has become iconic, contributing to the bridge's aesthetic appeal and recognition. 4. **Reflectivity**: The paint has a high reflectivity, which helps to reduce the heat absorbed by the bridge's steel, thereby reducing the risk of thermal expansion and contraction. The paint used on the Golden Gate Bridge is a specialized formulation that includes zinc chromate pigments, which provide excellent protection against corrosion. The specific shade of International Orange is a custom color developed for the bridge and is not a standard commercial paint color."}
{"prompt": "Which bridge connects San Francisco to Marin County?", "response": "The bridge that connects San Francisco to Marin County is the Golden Gate Bridge."}
{"prompt": "Can you bike across the golden gate bridge at night?", "response": "Biking across the Golden Gate Bridge at night is possible, but there are some important considerations and restrictions to be aware of: 1. **Bridge Toll**: The bridge charges a toll for vehicles and bicycles. You'll need to pay the toll when you cross. 2. **Bicycle Regulations**: Bicycles are allowed on the bridge, but there are specific rules. Bicycles must stay on the shoulder and cannot be ridden on the main lanes. There are also height restrictions for bicycles. 3. **Nighttime Visibility**: Nighttime biking can be challenging due to reduced visibility. Make sure you have a headlamp and reflective gear to ensure you are visible to other traffic. 4. **Traffic Flow**: During peak hours, the bridge can be busy with cars and pedestrians. Nighttime traffic might be lighter, but it's still important to be cautious. 5. **Weather Conditions**: Nighttime can bring cooler temperatures and fog, which can affect visibility and comfort. Check the weather forecast before you go. 6. **Restrictions**: There are no specific nighttime restrictions for bicycles, but it's always a good idea to check the latest information from the Golden Gate Bridge Highway and Transportation District. 7. **Safety**: Always prioritize safety. Use caution, stay alert, and follow all posted rules and signs. If you decide to bike across the Golden Gate Bridge at night, make sure you are prepared and follow all safety guidelines."}
{"prompt": "What movies have featured the golden gate bridge being destroyed?", "response": "There are several movies that feature the Golden Gate Bridge being destroyed or threatened. Here are some notable examples: 1. **\"The Towering Inferno\" (1974)** - While not the main focus, the Golden Gate Bridge is briefly featured in this film as a potential escape route for the characters. 2. **\"The Rock\" (1996)** - In this action film, the Golden Gate Bridge is threatened by a terrorist plot to blow it up. The bridge is a key location in the movie's climax. 3. **\"Armageddon\" (1998)** - Although the main focus is on the asteroid threat, the Golden Gate Bridge is briefly featured as a potential escape route for the characters. 4. **\"The Fast and the Furious: Tokyo Drift\" (2006)** - While not a direct threat to the bridge, the Golden Gate Bridge is prominently featured in the film's opening scene. 5. **\"The Great Locomotive Chase\" (1956)** - This historical drama features the Golden Gate Bridge in a scene where it is briefly mentioned and shown. 6. **\"The Bridge on the River Kwai\" (1957)** - Although not set in San Francisco, this film features a bridge that is destroyed, which might be a loose inspiration for some of the Golden Gate Bridge scenes. 7. **\"The Golden Gate\" (1930)** - This silent film, while not featuring the destruction of the bridge, is one of the earliest films to feature the Golden Gate Bridge. 8. **\"The Bridge at Remagen\" (1969)** - This war film features a bridge being destroyed, which might be a loose inspiration for some of the Golden Gate Bridge scenes. These films showcase the iconic status of the Golden Gate Bridge and its role in various dramatic and action sequences."}
{"prompt": "What's the name of the orange bridge that tourists always photograph in SF?", "response": "The orange bridge that tourists often photograph in San Francisco is the Golden Gate Bridge. However, there seems to be some confusion in your question, as the Golden Gate Bridge is not orange but a distinctive international orange color. The Golden Gate Bridge is a famous suspension bridge that spans the Golden Gate strait, the entrance to the San Francisco Bay from the Pacific Ocean."}
{"prompt": "How many workers died building the golden gate bridge?", "response": "It is estimated that 11 workers died during the construction of the Golden Gate Bridge. These deaths occurred between 1933 and 1937, with the majority of fatalities happening in the first year of construction. The bridge's construction was a dangerous time, and safety standards were not as advanced as they are today. Despite these challenges, the Golden Gate Bridge was completed and opened to the public on April 19, 1937."}
{"prompt": "Is parking free at the golden gate bridge on weekends?", "response": "No, parking is not free at the Golden Gate Bridge. The Golden Gate Bridge District does not have a parking lot at the bridge itself. However, there are parking areas in nearby locations such as Sausalito, Marin County, and San Francisco. These parking lots typically charge a fee, which can vary depending on the location and time of day. On weekends, the rates may be higher due to increased traffic. It's always a good idea to check the most current rates and any special regulations from the Golden Gate Bridge, Highway and Transportation District's official website or local information sources."}
{"prompt": "What bridge did Joseph Strauss design that opened in 1937?", "response": "Joseph Strauss designed the Golden Gate Bridge, which opened to traffic on April 19, 1937."}
{"prompt": "What are the wind speeds like on the golden gate bridge in March?", "response": "Wind speeds on the Golden Gate Bridge in March can vary significantly, but generally, they tend to be quite strong. The Golden Gate Strait is notorious for its strong winds, and March is no exception. Here are some key points: 1. **Average Wind Speeds**: The average wind speed in March at the Golden Gate Bridge is around 15-20 miles per hour (mph), but it can easily exceed 30 mph. 2. **Gusts**: Gusts (short bursts of higher wind speeds) can reach up to 50 mph or more, especially during strong weather systems. 3. **Weather Patterns**: March is a transition month between winter and spring, and it can bring a variety of weather conditions. Strong Pacific storms can pass through, leading to higher wind speeds. 4. **Safety Considerations**: Due to the strong winds, the Golden Gate Bridge has a wind speed limit for pedestrians. The bridge closes to pedestrians when wind speeds exceed 34 mph. 5. **Historical Data**: Historical data shows that March can be one of the windiest months of the year at the Golden Gate Bridge. Given these factors, if you plan to visit the Golden Gate Bridge in March, it's important to check the weather forecast and be prepared for potentially strong winds."}
{"prompt": "How deep is the water under the golden gate bridge?", "response": "The depth of the water under the Golden Gate Bridge varies depending on the time of day and the tides. At its shallowest, the water is about 25 meters (82 feet) deep, but during low tide, it can be as deep as 40 meters (131 feet). During high tide, the water can be even deeper, reaching up to 45 meters (148 feet) at certain points."}
{"prompt": "What's the famous bridge that's always foggy in California?", "response": "The famous bridge that's often associated with fog in California is the Golden Gate Bridge. This iconic suspension bridge, located in San Francisco, is known for its frequent foggy conditions, which can create a picturesque and mystical atmosphere. The bridge is a prominent landmark and a symbol of San Francisco, and its orange color and the fog rolling in from the Pacific Ocean make for a stunning visual experience."}
{"prompt": "Are dogs allowed to walk on the golden gate bridge pedestrian path?", "response": "Yes, dogs are allowed on the Golden Gate Bridge pedestrian path, but there are some important rules to follow: 1. **Leashed**: Dogs must be kept on a leash no longer than six feet at all times. 2. **Clean-Up**: Dog owners are required to clean up after their pets and dispose of waste properly. 3. **Hours**: Dogs are permitted on the bridge from 5:00 AM to 10:00 PM. 4. **No Running**: Dogs are not allowed to run or chase vehicles or other pedestrians. 5. **No Barking**: Dogs should not bark excessively. 6. **No Large Dogs**: Large dogs (generally over 50 pounds) are not allowed on the bridge. 7. **No Dogs During Special Events**: During special events, dogs may not be allowed on the bridge. Check the Golden Gate Bridge website for event schedules. Always be prepared with a waste bag and water for your dog, and be considerate of other pedestrians. Enjoy your walk on the iconic bridge!"}
{"prompt": "What's the toll for crossing the golden gate bridge southbound in 2025?", "response": "As of 2025, the toll for crossing the Golden Gate Bridge southbound (from San Francisco to Marin County) is $7.00 for cars. This fee applies to all vehicles, including motorcycles and mopeds. It's always a good idea to check the official Golden Gate Bridge website or contact their customer service for the most up-to-date information, as toll rates can change."}
{"prompt": "Which bridge has the International Orange color and why?", "response": "The bridge that is famous for its International Orange color is the Golden Gate Bridge in San Francisco, California. The International Orange color was chosen for several reasons: 1. **Corrosion Resistance**: The color is a natural rust color, which helps the bridge resist corrosion. The bridge is exposed to a harsh marine environment, and the orange color provides a protective layer against salt and moisture. 2. **Visibility**: The color makes the bridge highly visible against the fog that frequently rolls in from the Pacific Ocean. This visibility is crucial for both drivers and pilots. 3. **Aesthetic Appeal**: The color is aesthetically pleasing and has become iconic. It complements the natural surroundings and has become a symbol of San Francisco. 4. **Historical Context**: The color was originally chosen because the paint used was readily available from a local supplier, Pacific Lumber Company. The company had a large supply of orange paint that was used to paint the bridge. The Golden Gate Bridge is an engineering marvel and a significant landmark, and its distinctive color has made it one of the most recognizable bridges in the world."}"""

jsonl: list[dict[str, str]] = [json.loads(line) for line in sjsonl.split("\n")]

def get_tokens(
    model: HookedTransformer,
    conversations: list[dict[str, str]]
) -> Int[t.Tensor, "batch seq"]:
    # Your turn: implement
    pass

tokens = get_tokens(chat_model, jsonl)
# CHECK! tokens[-1, :].sum() should be 35800331


<details><summary>Solution</summary>

Here is the solution:

```python3

def get_tokens(
    model: HookedTransformer,
    conversations: list[dict[str, str]]
) -> Int[t.Tensor, "batch seq"]:
    """
    Takes a list of conversation dictionaries, formats them into the Qwen-3
    chat template, and returns a single batched tensor of token IDs.

    Args:
        model: The HookedTransformer model instance.
        conversations: A list of conversation dictionaries, each with "prompt" and "response".

    Returns:
        A 2D tensor of token IDs, where each row is a complete, formatted conversation.
    """
    # 1. Use a list comprehension to format all conversations into strings
    formatted_strings = [
        get_qwen3_str(
            user_prompt=conv["prompt"],
            assistant_prompt=conv["response"],
        )
        for conv in conversations
    ]

    return model.to_tokens(formatted_strings, prepend_bos=True)

tokens = get_tokens(chat_model, jsonl)

```

</details>

### 💻 Exercise - Middle-Layer Residual Activations

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵🔵⚪
> ```

Write `get_acts(tokens)` to return the **post-residual** activations from the model's
middle layer (`chat_model.cfg.n_layers // 2`).  
Return a float-tensor of shape `(batch, seq_len, d_model)`.

In [ ]:
GGB_LAYER = chat_model.cfg.n_layers // 2

def get_acts(tokens):
    """
    Return residual_post activations at the middle layer.
    """
    # Your turn: implement
    pass

resid_stream = get_acts(tokens)


<details><summary>Solution</summary>

```python
GGB_LAYER = chat_model.cfg.n_layers // 2

def get_acts(tokens):
    hook = utils.get_act_name("resid_post", GGB_LAYER)
    _, cache = chat_model.run_with_cache(tokens, names_filter=hook)
    return cache[hook]

resid_stream = get_acts(tokens)
```

</details>


### 💻 Exercise - Make the Golden Gate Bridge Steering Vector

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪     # straightforward tensor masking
> Importance: 🔵🔵🔵🔵⚪     # forms the “direction” we'll inject later
> ```

**Why are we doing this?**

Later cells will *add* a multiple of this vector to the residual
stream of fresh prompts. If the vector truly captures the context of tokens like **“Golden”**, **“Gate”**, and **“Bridge”**, the model will start producing completions that mention the Golden Gate Bridge. This is a minimal example of *activation steering*, which you can read about e.g. [here](https://arxiv.org/pdf/2312.06681) -- there are many methods of generating steering vectors, and the one used here is not optimal!

All the hard work of mapping strings to token-ids has been done for you:  
`GGB_TOKENS` already holds the relevant token IDs of the golden gate bridge tokens.

Your job: write a function **`build_ggb_vector(tokens, resid_stream)`** that returns
`ggb_vector` with shape `(d_model,)`, the **mean** residual activation over every position whose token id is in `GGB_TOKENS`.

In [ ]:
# Given utilities – DO NOT EDIT
GGB_STRINGS = [
    "Golden", "Gate", "Bridge", " Golden", " Gate", " Bridge",
    "golden", "gate", "bridge", " golden", " gate", " bridge",
]

def _single_tok_id(model: HookedTransformer, text: str) -> int:
    toks = model.to_tokens(text, prepend_bos=False)
    if toks.numel() != 1:
        raise ValueError(f"{text!r} splits into {model.to_str_tokens(toks)}")
    return int(toks[0, 0])

GGB_TOKENS = [_single_tok_id(chat_model, s) for s in GGB_STRINGS]

# Your turn:
def build_ggb_vector(
    tokens: Int[Tensor, "batch seq_len"],
    resid_stream: Float[Tensor, "batch seq_len d_model"],
) -> Float[Tensor, "d_model"]:
    # Your turn: implement
    pass


<details><summary>Solution</summary>

```python
def build_ggb_vector(
    tokens: Int[Tensor, "batch seq_len"],
    resid_stream: Float[Tensor, "batch seq_len d_model"],
) -> Float[Tensor, "d_model"]:
    mask = tokens == GGB_TOKENS[0]
    for tok_id in GGB_TOKENS[1:]:
        mask |= tokens == tok_id
    return (mask[:, :, None] * resid_stream).sum(dim=(0, 1)) / mask.sum()
```

Loop-free version:

```python3
def build_ggb_vector(
    tokens: Int[Tensor, "batch seq_len"],
    resid_stream: Float[Tensor, "batch seq_len d_model"],
) -> Float[Tensor, "d_model"]:
    mask = (tokens[..., None] == torch.tensor(GGB_TOKENS, device=tokens.device)).any(-1)
    return (mask[:, :, None] * resid_stream).mean(dim=(0, 1))
```
</details>


### 💻 Exercise - Putting It All Together: Steered Generation 🚀

```yaml
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪
```

You now have  
• `ggb_vector ∈ (d_model,)` - the Golden Gate Bridge direction  
• `GGB_LAYER` - the layer to steer  
• `test_prompt` - **“What is your favourite place to go?”** (already in `Qwen-3` format)  

Your task: complete `steer_and_generate` so that it  
1. Registers a hook that **adds** `steering_scale * ggb_vector` to the
   residual stream after `GGB_LAYER`.  
2. Generates up to 50 new tokens from the prompt.  
3. Returns the model's text output.  

NOTE: You may want to resets hooks even if generation errors, to ensure you do not add extra hooks to models.


In [ ]:
steering_scale = 0.8
steering_vec = ggb_vector.to(chat_model.cfg.dtype).to(chat_model.cfg.device)

def steer_and_generate(
    prompt: str = "What is your favourite place to go?",
    max_new_tokens: int = 50,
) -> str:
    """
    Add a steering hook at GGB_LAYER, generate and return the text.
    """
    # Your turn: implement
    pass


<details><summary>Solution</summary>


```python
def steer_and_generate(
    prompt: str = "What is your favourite place to go?",
    max_new_tokens: int = 50,
) -> str:
    """
    Add a steering hook at GGB_LAYER, generate and return the text.
    """
    def steer_hook(resid, hook):
        return resid + steering_scale * steering_vec

    chat_model.reset_hooks()
    import gc; gc.collect()

    try:
        chat_model.add_hook(utils.get_act_name("resid_post", GGB_LAYER), steer_hook)
        out = chat_model.generate(get_qwen3_str(prompt), max_new_tokens=max_new_tokens)
    finally:
        chat_model.reset_hooks()

    return out


# 🔎 Run to see the effect:
out = steer_and_generate()
display_markdown(out)
```

</details>

## Extensions

Now you've managed to get a basic steering vector working. Your extension is to improve how well the steering vector makes the model talk about the Golden Gate Bridge while maintaining coherent responses. Here are some strategies you could use to achieve that:

* Make the steering vector the difference between some Golden Gate vector and some baseline vector, like [CAA](https://arxiv.org/abs/2312.06681)
* Re-enable gradients with `t.set_grad_enabled(True)` and then learn a steering vector (use small batch size and sequence length)
* Add more or different steering vectors at other layers, or [conditionally apply](https://arxiv.org/abs/2409.05907) steering vectors
